# Setup and basic objects

Get started with EnOSlib on Grid'5000.

---

- Website: https://discovery.gitlabpages.inria.fr/enoslib/index.html
- Instant chat: https://framateam.org/enoslib
---

This is the first notebooks of a series that will let you discover the main features of EnOSlib/Grid'5000.

If you want to actually execute them you'll need to setup your environment properly. 
We sum up here the different step to achieve this process.

1. Get a Grid'5000 account
    - Register using this [page](https://www.grid5000.fr/w/Grid5000:Get_an_account). 
      Pay attention to the fact that uploading a SSH key (public part) is mandatory to perform any EnOSlib action.
    - Make sure the SSH connection is ready. You can follow this [tutorial](https://www.grid5000.fr/w/Getting_Started).
2. Install EnOSlib
    - Follow the steps [here](https://discovery.gitlabpages.inria.fr/enoslib/tutorials/grid5000.html#installation).


## Testing the import

In [2]:
import enoslib as en

## Resources abstractions

In this notebook we won't execute anything remotely, instead we'll just cover some basic abstractions provided by the library.
We start with the abstractions of the resources (machines and networks that are usually givn by an infrastructure)


### Host

An host is anything we can connect to and act on it. Most of the time it corresponds to a machine reachable through SSH.
The datastructure reflects this.

Usually you don't instantiate hosts manually but they are brought to you by EnOSlib (because they depend on a scheduler decision).


In [18]:
bare_host = en.Host("192.168.0.1")
host_with_alias = en.Host("192.168.0.2", alias="one_alias")
host_with_alias_and_username = en.Host("192.168.0.3", alias="one_alias", user="foo")

In [19]:
bare_host

Host(address='192.168.0.1', alias='192.168.0.1', user=None, keyfile=None, port=None, extra={}, net_devices=set())

In [20]:
host_with_alias

Host(address='192.168.0.2', alias='one_alias', user=None, keyfile=None, port=None, extra={}, net_devices=set())

In [21]:
host_with_alias_and_username

Host(address='192.168.0.3', alias='one_alias', user='foo', keyfile=None, port=None, extra={}, net_devices=set())

The local machine can be represented by an instance of the `LocalHost` object. This is a specialization of an `Host`, the connection to this host will be made using sub-processes (instead of SSH). We can see it in the `extra` attribute of the `LocalHost` object. This `extra` attribute is actually interpreted when we won't to do some "remote" actions on our hosts.

In [22]:
localhost = en.LocalHost()
localhost

LocalHost(address='localhost', alias='localhost', user=None, keyfile=None, port=None, extra={'ansible_connection': 'local'}, net_devices=set())

Other types of Hosts are possible. The library has a `DockerHost` which represents a docker container we want to reach using the docker line protocol. One needs to specify where this container is running by passing an host instance.

In [29]:
docker_host = en.DockerHost("alias", "container_name", host_with_alias_and_username)
docker_host

DockerHost(address='container_name', alias='alias', user='foo', keyfile=None, port=None, extra={'ansible_connection': 'docker', 'ansible_docker_extra_args': '-H ssh://foo@192.168.0.3', 'mitogen_via': 'foo@192.168.0.3'}, net_devices=set())

### Roles

During an experiment your hosts will serve different purposes: some will host the system you are studying while other will install third party tools to inject some load, observe ...

A natural way of configuring differently several sets of hosts is to tag them and group them according to their tags.

The `Roles` datastructure serves this purpose: it lets you group your hosts based on tags. It follow a `dict-like` interface.

In [38]:
h1 = en.Host("10.0.0.1")
h2 = en.Host("10.0.0.2")
h3 = en.Host("10.0.0.3")
roles = en.Roles()
roles["tag1"] = [h1, h2]
roles["tag2"] = [h3]
roles["tag3"] = [h2, h3]

roles

{'tag1': [Host(address='10.0.0.1', alias='10.0.0.1', user=None, keyfile=None, port=None, extra={}, net_devices=set()), Host(address='10.0.0.2', alias='10.0.0.2', user=None, keyfile=None, port=None, extra={}, net_devices=set())], 'tag2': [Host(address='10.0.0.3', alias='10.0.0.3', user=None, keyfile=None, port=None, extra={}, net_devices=set())], 'tag3': [Host(address='10.0.0.2', alias='10.0.0.2', user=None, keyfile=None, port=None, extra={}, net_devices=set()), Host(address='10.0.0.3', alias='10.0.0.3', user=None, keyfile=None, port=None, extra={}, net_devices=set())]}

### Network and Networks

`Network` and `Networks`  are the same as `Host` and `Roles` but for networks:
- `Network` represent a single Network
- `Networks` represent a "Roles" of Network: networks indexed by their tags
.

Networks are usually given by an infrastructure and thus you won't really instantiate `Network` nor `Networks` by yourself.
More precisely there exists a specific subclass of `Network` per infrastructure which will be returned automatically by EnOSlib when needed.

Moreover `Network` datastructure isn't exposed in EnOSlib at the top level, let's see however how a `DefaultNetwork` can look like. A `DefaultNetwork` is a very common abstraction of a network that allows to represent a basic network with optionnally a pool of free ips/macs address. For instance a subnet or a vlan on Grid5000 are represented by a specific `DefaultNetwork`.

In [39]:
from enoslib.objects import DefaultNetwork

In [48]:
one_network = DefaultNetwork("192.168.1.0/24")
one_network_with_a_pool_of_ips = DefaultNetwork("192.168.1.0/24", ip_start="192.168.1.10", ip_end="192.168.1.100") 

In [49]:
one_network

In [50]:
one_network_with_a_pool_of_ips

In [54]:
# get one free ip
ip_gen = one_network_with_a_pool_of_ips.free_ips
next(ip_gen)

IPv4Address('192.168.1.10')